In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid

/home/sachchida/anaconda3/envs/timeseries/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset=Planetoid(root='/tmp/Cora', name='Cora')
print(dataset)

Cora()


Processing...
/home/sachchida/anaconda3/envs/timeseries/lib/python3.11/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  out = pickle.load(f, encoding='latin1')
/home/sachchida/anaconda3/envs/timeseries/lib/python3.11/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  out = pickle.load(f, encoding='latin1')
/home/sachchida/anaconda3/envs/timeseries/lib/python3.11/site-packages/torch_geometric/io/planetoid.py:107: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  out = pickle.load(f, enc

In [3]:
print("no of graphs:", len(dataset))
print("num features:", dataset.num_features)
print("num classes:", dataset.num_classes)
print("num of edges:", dataset[0].num_edges)
print("num of edge features:", dataset.num_edge_features)

no of graphs: 1
num features: 1433
num classes: 7
num of edges: 10556
num of edge features: 0


In [4]:
print("edge_index:", dataset.data.edge_index.shape)
print("x:", dataset.data.x.shape)
print("y:", dataset.data.y.shape)
print("train_mask:", dataset.data.train_mask.shape)
print("val_mask:", dataset.data.val_mask.shape)
print("test_mask:", dataset.data.test_mask.shape)
print("num nodes:", dataset.data.num_nodes)

edge_index: torch.Size([2, 10556])
x: torch.Size([2708, 1433])
y: torch.Size([2708])
train_mask: torch.Size([2708])
val_mask: torch.Size([2708])
test_mask: torch.Size([2708])
num nodes: 2708


/tmp/ipykernel_16973/792315066.py:1: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  print("edge_index:", dataset.data.edge_index.shape)
/tmp/ipykernel_16973/792315066.py:2: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  print("x:", dataset.data.x.shape)
/tmp/ipykernel_16973/792315066.py:3: UserWarning: It is not recommended to directly access the internal storage format `data

In [5]:
print(dataset.data.edge_index)

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


/tmp/ipykernel_16973/135294416.py:1: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  print(dataset.data.edge_index)


In [6]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [7]:
data=dataset[0]  # Get the first graph object.

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, dataset.num_classes,aggr='max')
        # self.conv2 = SAGEConv(16, dataset.num_classes)

    def forward(self):
        x=self.conv1(data.x, data.edge_index)

        return F.log_softmax(x, dim=1)

In [15]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model,data=Net().to(device),data.to(device)
data=data.to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [16]:
def train():
    model.train()
    optimizer.zero_grad()
    out=model()
    loss=F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

In [17]:
def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [19]:
best_val_acc = test_acc = 0
for epoch in range(1, 501):
    loss = train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, '
              f'Test Acc: {tmp_test_acc:.4f}')

Epoch: 010, Loss: 0.0063, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7330
Epoch: 020, Loss: 0.0063, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7330
Epoch: 030, Loss: 0.0062, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7340
Epoch: 040, Loss: 0.0062, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7330
Epoch: 050, Loss: 0.0061, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7360
Epoch: 060, Loss: 0.0061, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7340
Epoch: 070, Loss: 0.0060, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7360
Epoch: 080, Loss: 0.0060, Train Acc: 1.0000, Val Acc: 0.7140, Test Acc: 0.7380
Epoch: 090, Loss: 0.0059, Train Acc: 1.0000, Val Acc: 0.7140, Test Acc: 0.7400
Epoch: 100, Loss: 0.0059, Train Acc: 1.0000, Val Acc: 0.7140, Test Acc: 0.7410
Epoch: 110, Loss: 0.0059, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7420
Epoch: 120, Loss: 0.0058, Train Acc: 1.0000, Val Acc: 0.7160, Test Acc: 0.7420
Epoch: 130, Loss: 0.0058, Train Acc: 1.0000, Val Acc